# 📊 Analyzing Software Issue Tracking and Resolution Trends: Insights from Apache JIRA

This project analyzes issue tracking, resolution patterns, and changelog activity using a public Apache JIRA dataset. Leveraging PySpark, we examine issue lifecycles, team collaboration, and resolution times. The analysis includes engineered features, grouping strategies, and filtering based on time (last two years) to generate valuable insights.

---
## 🧹 Data Preprocessing
The dataset consists of three main components:
- `issues.csv`: metadata about each JIRA issue.
- `changelog.csv`: tracks status changes and field updates.
- `comments.csv`: discussion history tied to each issue.

We performed the following preprocessing steps:
- Cleaned column names by removing dots/spaces.
- Dropped duplicates based on unique identifiers (`key`, `comment_id`, `id`).
- Removed rows with nulls in required columns.
- Filtered malformed timestamps (e.g., those not starting with '20').

---
## 🧮 Feature Engineering
We enriched the data with:
- `comment_count`: Number of comments per issue.
- `status_change_count`: Number of status field changes per issue.
- `resolution_days`: Time taken (in days) to resolve an issue.

Only issues created in the last **2 years** and with valid `resolution_days` were retained for analysis.


## 📈 Q1: What is the distribution of issue resolution times for the last two years?

By filtering issues created in the last 24 months and calculating `resolution_days`, we isolate a subset of recent issues for more relevant analysis. This allows us to:
- Avoid outdated project workflows influencing results.
- Focus on modern triage patterns and prioritization practices.

This cleaned and scoped dataset is exported for further visualization.

---
## 📊 Q2: How does resolution time vary across issue priorities and statuses?

We grouped the dataset by `priority_name` and `status_name`, calculating the average resolution time (`avg_resolution_days`) for each combination.

### 🔍 Interpretation:
- High-priority issues (e.g., Blocker, Critical) are generally expected to resolve faster.
- However, variations within statuses (like “In Progress” or “Review”) reveal inefficiencies or bottlenecks.
- Average resolution times can identify whether priorities are enforced correctly or overlooked during execution.

This result is exported for visualization via Tableau to generate comparative stacked bars.


## 🔄 Q3: What are the top 10 most frequently changed fields in JIRA issues?

We analyzed the `changelog.csv` to determine:
- Fields with the **most total changes**.
- How many **unique issues** were affected per field.

### 📌 Insights:
- High change frequency in fields like `status`, `priority`, or `assignee` can indicate process fluidity or inconsistency.
- This helps identify areas where workflow policies may need to be refined or automated.

The results were exported for additional reporting and graph generation.


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import unix_timestamp, col, round, current_timestamp, expr, to_timestamp
from pyspark.sql import DataFrame

# Create Spark session
spark = SparkSession.builder.appName("ApacheJiraML").getOrCreate()

# Fix for Spark 3+ timestamp parsing issues
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

# Function to clean column names
def clean_column_names(df: DataFrame) -> DataFrame:
    for col_name in df.columns:
        clean_name = col_name.replace(".", "_").replace(" ", "_")
        if clean_name != col_name:
            df = df.withColumnRenamed(col_name, clean_name)
    return df

# Load CSVs from HDFS
issues_df = spark.read.option("header", True).option("inferSchema", True).option("multiLine", True).option("escape", "\"").csv("/user/your_username/Apache_JIRA_Issues/issues.csv")
changelog_df = spark.read.option("header", True).option("inferSchema", True).option("multiLine", True).option("escape", "\"").csv("/user/your_username/Apache_JIRA_Issues/changelog.csv")
comments_df = spark.read.option("header", True).option("inferSchema", True).option("multiLine", True).option("escape", "\"").csv("/user/your_username/Apache_JIRA_Issues/comments.csv")

# Clean column names
issues_df = clean_column_names(issues_df)
changelog_df = clean_column_names(changelog_df)
comments_df = clean_column_names(comments_df)

# Deduplicate
issues_df = issues_df.dropDuplicates(["key"])
comments_df = comments_df.dropDuplicates(["comment_id"])
changelog_df = changelog_df.dropDuplicates(["id"])

# Drop rows with nulls in required columns
issues_df = issues_df.dropna(subset=["key", "created", "updated", "status_name", "priority_name", "issuetype_name"])
comments_df = comments_df.dropna(subset=["key", "comment_id"])
changelog_df = changelog_df.dropna(subset=["key", "field", "id"])

# Join comment count and status change count
comment_counts = comments_df.groupBy("key").agg(F.count("comment_id").alias("comment_count"))
status_change_counts = changelog_df.filter(col("field") == "status").groupBy("key").agg(F.count("id").alias("status_change_count"))

issues_df = issues_df.join(comment_counts, "key", "left").join(status_change_counts, "key", "left") \
    .fillna({"comment_count": 0, "status_change_count": 0})

# 🔍 Filter out malformed timestamps (e.g., starting with 2-digit year like '12-09-17')
issues_df = issues_df.filter(col("created").startswith("20")).filter(col("updated").startswith("20"))

# Convert to timestamp
issues_df = issues_df \
    .withColumn("created", to_timestamp("created", "yyyy-MM-dd HH:mm:ss")) \
    .withColumn("updated", to_timestamp("updated", "yyyy-MM-dd HH:mm:ss"))


#What is the distribution of issue resolution times for last two years?
# Calculate resolution_days
issues_df = issues_df.withColumn(
    "resolution_days",
    round((unix_timestamp("updated") - unix_timestamp("created")) / 86400, 2)
)

# Filter: resolved issues in last 2 years
two_years_ago = expr("add_months(current_timestamp(), -24)")
resolved_df = issues_df.filter(
    (col("resolution_days").isNotNull()) & (col("created") >= two_years_ago)
)

# Preview
resolved_df.select(
    "key", "created", "updated", "resolution_days",
    "project_name", "status_name", "priority_name", "issuetype_name"
).show(5)


"""
# ✅ Export to HDFS
# Select columns for export
export_df = resolved_df.select(
    "key",
    "created",
    "updated",
    "resolution_days",
    "project_name",
    "status_name",
    "priority_name",
    "issuetype_name",
    "comment_count",
    "status_change_count"
)


# Export as single CSV with header for visualization
export_df.coalesce(1).write.mode("overwrite").option("header", True).csv("/user/npatida/output/resolved_df_q1")
"""


# Question 2 How does resolution time vary across issue priorities and statuses?
from pyspark.sql.functions import avg

# Group by priority and status to get average resolution time
priority_status_avg_df = resolved_df.groupBy("priority_name", "status_name") \
    .agg(avg("resolution_days").alias("avg_resolution_days")) \
    .orderBy("priority_name", "status_name")

priority_status_avg_df.show(5)

"""
# Save to HDFS for Tableau or Excel
priority_status_avg_df.coalesce(1).write.mode("overwrite").option("header", True) \
    .csv("/user/npatida/output/q2_priority_status_resolution")
"""

# Q3: Top 10 fields changed most frequently + how many unique issues were affected
changelog_summary_df = changelog_df.groupBy("field") \
    .agg(
        F.count("*").alias("total_changes"),
        F.countDistinct("key").alias("affected_issues")
    ) \
    .orderBy(F.desc("total_changes")) \
    .limit(10)

# Show the summary
changelog_summary_df.show(truncate=False)

# Save to HDFS for reporting
changelog_summary_df.coalesce(1).write.mode("overwrite").option("header", True) \
    .csv("/user/your_username/output/q3_field_change_summary")

